In [26]:
import pandas as pd
import numpy as np
import math
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns

In [2]:
# Display Pipeline
from sklearn import set_config
set_config(display='diagram')


In [3]:
def imprimir_metricas (y, y_pred, dataset_name='valid'):
  mean_y = np.mean(y)
  mean_pred = np.mean(y_pred)

  rmse = round(math.sqrt(mean_squared_error(y, y_pred)), 2)
  mae = mean_absolute_error(y, y_pred)
  r2 = r2_score(y, y_pred)
  mape = mean_absolute_percentage_error(y, y_pred)
  
  report = pd.DataFrame({
    'metric': ['RMSE', 'MAE', 'R^2', 'MAPE', 'Avg. target', 'Avg. Prediction'],
    dataset_name: [rmse, mae, r2, mape, mean_y, mean_pred]
  })

  report[dataset_name] = report[dataset_name].round(2)
  return report

In [50]:
df = pd.read_csv('Base_EMBRAESP_trabalhada.csv')

In [51]:
df.columns

Index(['Unnamed: 0.1', 'ID', 'TIPO_EMP', 'MES_LAN', 'ANO_LAN', 'DATA_ENT',
       'DIST', 'SUBPREF', 'MUNICIPIO', 'TIPO_VIA', 'ZONA', 'DORM_UNID',
       'BANH_UNID', 'GAR_UNID', 'ELEV', 'COB', 'BLOCOS', 'UNIDAND', 'ANDARES',
       'AR_UT_UNID', 'AR_TT_UNID', 'AR_TT_TERR', 'TT_UNID', 'DORM_EMP',
       'BANH_EMP', 'GAR_EMP', 'AU_EMP', 'AT_EMP', 'PC_TT_UN', 'PC_M2_AU',
       'PC_M2_AT', 'PC_TT_ATU', 'PC_AU_ATU', 'PC_AT_ATU', 'PC_EMP_ATU',
       'VLR_US__CO', 'PC_TT_UN_U', 'PC_M2_AU_U', 'PC_M2_AT_U', 'SIST_FINAN',
       'AGENTE', 'INCORPORAD', 'CONSTRUTOR', 'INCORPOR_A', 'CONSTRUT_A',
       'COOPERATIV', 'HOTEL', 'FLAT', 'EXFLAT', 'AP2010', 'SC_2010',
       'RENRESP91', 'RENRESP00', 'RENRESP10', 'PCMEDAU91', 'PCMEDAU00',
       'PCMEDAU10', 'BANH_DORM', 'Unnamed: 0', 'Domicílios_Setor',
       'Média_Moradores_por_domicilio_setor', 'Média_Renda_Responsaveis_Setor',
       'Domicilios no setor censitário', 'Domicilios alugados',
       'Domicilios unipessoais', 'Domicilios 2 ou 3 mo

In [5]:
df.shape

(16935, 87)

In [52]:
target = 'PC_AU_ATU_log'
numericas = ['ANO_LAN', 'DORM_UNID', 'BANH_UNID', 'GAR_UNID']
categoricas = ['REGIAO']
features = numericas + categoricas
#['', 'ANO_LAN', 'DORM_UNID', 'BANH_UNID', 'GAR_UNID']
v_ibge = ['Total de filhos dos responsáveis de domicílios', 'PERC_ALUGADOS',
       'PERC_UNIPESSOAIS', 'PERC_2_3_MORADORES', 'PERC_BRANCOS', 'PERC_PPI']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(target, axis=1), df[target], test_size=0.2)

In [54]:
preprocessing = ColumnTransformer(transformers=[
    ("cat", OneHotEncoder(), categoricas),
    #("num", SimpleImputer(), v_ibge)
    ], remainder='passthrough')

In [55]:
#PIPELINE DEFAULT
model = Pipeline(steps=[
    ('preprocessamento', preprocessing),
    ('regressao', RandomForestRegressor())
])

In [56]:
model.fit(X_train[features], y_train)

Pipeline(steps=[('preprocessamento',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['REGIAO'])])),
                ('regressao', RandomForestRegressor())])

In [57]:
model.score(X_test, y_test)

0.669557673259138

Conjunto 1: 0.65 (RandomF), 0.39 (RegressãoL)
Conjunto IBGE: 0.53 (RandomF), 0.26 (RegressãoL)
Conjunto 1_log: 0.69 (RandomF), 0.40 (RegressãoL)

In [82]:
y_pred = model.predict(X_test)

In [83]:
baseline_report = imprimir_metricas(y_test, y_pred)
baseline_report

,metric,valid
0,RMSE,1403.41
1,MAE,1011.03
2,R^2,0.66
3,MAPE,0.20
4,Avg. target,5455.46
5,Avg. Prediction,5478.53


In [58]:
reg_linear1 = sm.ols(formula='PC_AU_ATU_log ~ REGIAO + ANO_LAN + DORM_UNID + BANH_UNID + GAR_UNID', data=df.iloc[X_train.index,:]).fit()
reg_linear1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          PC_AU_ATU_log   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     707.6
Date:                Thu, 02 Mar 2023   Prob (F-statistic):               0.00
Time:                        19:53:55   Log-Likelihood:                -5414.3
No. Observations:               13442   AIC:                         1.086e+04
Df Residuals:                   13428   BIC:                         1.096e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               19.7993      0.784     25.241      0.000      18.262      21.337
REGIAO[T.Centro-Sul]    -0.0110      0.016     -0.668      0.504      -0.043       0.021
REGIAO[T.Leste 1]       -0.5389      0.020    -27.524      0.000      -0.577      -0.500
REGIAO[T.Leste 2]       -0.6771      0.034    -19.993      0.000      -0.743      -0.611
REGIAO[T.Nordeste]      -0.2846      0.018    -15.422      0.000      -0.321      -0.248
REGIAO[T.Noroeste]      -0.5320      0.031    -17.187      0.000      -0.593      -0.471
REGIAO[T.Oeste]         -0.0033      0.016     -0.203      0.839      -0.035       0.028
REGIAO[T.RMSP]          -0.4964      0.016    -30.749      0.000      -0.528      -0.465
REGIAO[T.Sudeste]       -0.2725      0.017    -16.115      0.000      -0.306      -0.239
REGIAO[T.Sul]           -0.2981      0.019    -15.723      0.000      -0.335      -0.261
ANO_LAN                 -0.0055      0.000    -14.065      0.000      -0.006      -0.005
DORM_UNID               -0.1913      0.006    -30.917      0.000      -0.203      -0.179
BANH_UNID                0.1910      0.007     26.030      0.000       0.177       0.205
GAR_UNID                 0.0770      0.006     12.907      0.000       0.065       0.089
==============================================================================
Omnibus:                      119.791   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              164.865
Skew:                           0.120   Prob(JB):                     1.58e-36
Kurtosis:                       3.487   Cond. No.                     5.03e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.03e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""